# 嘗試用`langchain`讀取各種格式的檔案

## 安裝需要的套件

In [1]:
!pip install -r requirements.txt


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

## 讀一些pdf之類的東西

In [3]:
urls = [
    "https://fundreport.moneydj.com/FileOpenAPI.aspx?A=SPY&B=4&type=GetETFInfo1", # 英文pdf
    "https://www.yuantafunds.com/fund/download/1066%E5%8F%B0%E7%81%A3%E5%8D%93%E8%B6%8A50-%E5%85%AC%E9%96%8B%E8%AA%AA%E6%98%8E%E6%9B%B8.pdf",  # 中文pdf
    "https://i.imgur.com/QU8C5va.jpg",  # 中文jpg
    "https://www.mtsac.edu/webdesign/accessible-docs/word/example03.docx",  # 英文word
    "https://scholar.harvard.edu/files/torman_personal/files/samplepptx.pptx",  # 英文ppt
    "https://dornsife.usc.edu/assets/sites/298/docs/ir211wk12sample.xls",  # 英文excel
    "https://www.mohw.gov.tw/dl-70183-fdf20a0a-0d1b-467f-a1c8-3864523d3a08.html",  # 中文word
    "https://homepage.ntu.edu.tw/~karchung/Intro_to_ling/%ADp%BA%E2%BBy%A8%A5%BE%C7%AA%BA%C0%B3%A5%CE.ppt",  # 中文ppt
    "https://www.chyi.mohw.gov.tw/warehouse/%7B2F8E2C39-A6A9-4DBE-81D7-D9C452DA1C43%7D/%E5%B8%B8%E7%94%A8%E7%B8%AE%E5%AF%AB%E8%A1%A8.xls",  # 中文excel
]

用Unstructured讀

In [4]:
loader = UnstructuredURLLoader(urls=urls)
documents = loader.load()
len(documents)

convert /var/folders/lh/l56kxqzx1_71q78v4b44rm4h0000gn/T/tmp87r31um1 -> /private/var/folders/lh/l56kxqzx1_71q78v4b44rm4h0000gn/T/tmpdk82iwek/tmp87r31um1.pptx using filter : Impress Office Open XML


9

切割之後進vectordb

In [5]:
print("Splitting documents")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=10)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

print("Creating vectorstore")
db = Chroma.from_documents(texts, embeddings)

Splitting documents
Creating vectorstore


驗證看看有沒有找到東西

In [6]:
def query(q):
    docs = db.similarity_search(q)
    return docs[0].page_content, docs[0].metadata

print("中英文pdf:")
print("-------------------------------")
print(query("S&P 500 ETF sponsor"))
print("-------------------------------")
print(query("0050收益分配"))
print("-------------------------------")
print("")
print("中文jpg")
print("-------------------------------")
print(query("Toyota Camry"))
print("-------------------------------")
print(query("高級轎車"))
print("-------------------------------")
print("")
print("英文word/ppt/excel")
print("-------------------------------")
print(query("Word Documents Template"))
print("-------------------------------")
print(query("Sample PowerPoint File"))
print("-------------------------------")
print(query("SalesOrders Smith Desk"))
print("-------------------------------")
print("")
print("中文word/ppt/excel")
print("-------------------------------")
print(query("死亡證明書"))
print("-------------------------------")
print(query("計算語言學的工具資源與應用 高照明 台大外文系"))
print("-------------------------------")
print(query("內視鏡逆行性胰膽管攝影術"))
print("-------------------------------")


中英文pdf:
-------------------------------
('INVESTMENTS IN AFFILIATES OF THE TRUSTEE AND THE SPONSOR SPDR S&P 500® ETF Trust has invested in State Street Corp., which is considered an affiliate of the Trustee and Intercontinental Exchange,', {'source': 'https://fundreport.moneydj.com/FileOpenAPI.aspx?A=SPY&B=4&type=GetETFInfo1'})
-------------------------------
('第十五條 收益分配 第一項 本基金投資所得之現金股利、利息收 入、已實現盈餘配股之股票股利面額 部分、收益平準金、已實現資本利得 扣除資本損失(包括已實現及未實現 之資本損失)及本基金應負擔之各項 成本費用後，為可分配收益。 基金收益分配以當年度之實際可分 配收益餘額為正數方得分配。本基金 每受益權單位之可分配收益低於會 計年度結束日每受益權單位淨資產 價值百分之____，經理公司不予分', {'source': 'https://www.yuantafunds.com/fund/download/1066%E5%8F%B0%E7%81%A3%E5%8D%93%E8%B6%8A50-%E5%85%AC%E9%96%8B%E8%AA%AA%E6%98%8E%E6%9B%B8.pdf'})
-------------------------------

中文jpg
-------------------------------
('Example A:', {'source': 'https://www.mtsac.edu/webdesign/accessible-docs/word/example03.docx'})
-------------------------------
('1101 1216 1301 1303 1326 1402 1590 亞德客-KY 1605 2002 2207 2303 2308 2317 2327 2330 235

## 結論
1. 內建Unstructured套件針對中英文pdf效果都很好
2. 內建Unstructured套件對word/ppt/excel支援也很好
3. 對中文圖像辨識OCR支援不佳。需要再研究使用其他第三方套件。

# 中文OCR套件整合嘗試
在這個階段我們嘗試優化中文OCR套件。採用下列步驟嘗試優化
1. 嘗試使用用Unstructured內建的Tesseract套件的中文OCR支援。
2. 嘗試更換使用中文支援較好的[PaddleOCR](https://github.com/PaddlePaddle/PaddleOCR)做OCR

## 安裝Tesseract中文
https://www.simpleindex.com/knowledge-base/language-pack-for-standard-tesseract-ocr/

1. 下載`chi_tra.traineddata` - https://github.com/tesseract-ocr/tessdata/
2. 把下載的檔案加到`C:\Program Files (x86)\SimpleIndex\Tesseract\v3.04\tessdata`目錄 (Mac用homebrew安裝的路徑在 `/opt/homebrew/Cellar/tesseract/5.3.1/share/tessdata`)
3. 用`ocr_languages`參數指定`UnstructuredURLLoader`用繁體中文OCR

In [7]:
urls = [
    "https://i.imgur.com/QU8C5va.jpg",  # 中文jpg
]

loader = UnstructuredURLLoader(urls=urls, ocr_languages="chi_tra")
documents = loader.load()
len(documents)

print("Splitting documents")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=10)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

print("Creating vectorstore")
db = Chroma.from_documents(texts, embeddings)

print("中文jpg")
print("-------------------------------")
print(query("Toyota Camry"))
print("-------------------------------")
print(query("高級轎車"))
print("-------------------------------")


Splitting documents
Creating vectorstore


Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


中文jpg
-------------------------------
("﹒aHRSSS ﹍\n\ns ﹍\n\n@ ﹣ˍ﹍﹍@ @ 乙\n\n高 級 轎 車\n\n提 供 ﹁Toyota Camry 句 級 或 以 上 的 車 蓮 ， 經 濟 實 惠 ， 捨 乘 體 驗 依 然 舒 適 ， 是 機 場 接 造 最 受 歡 迎 的 服 務 車 著 。\n\n﹛_′﹛'籌7 1 穴 4 ˍ工盄 1 穴 4", {'source': 'https://i.imgur.com/QU8C5va.jpg'})
-------------------------------


Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


("﹒aHRSSS ﹍\n\ns ﹍\n\n@ ﹣ˍ﹍﹍@ @ 乙\n\n高 級 轎 車\n\n提 供 ﹁Toyota Camry 句 級 或 以 上 的 車 蓮 ， 經 濟 實 惠 ， 捨 乘 體 驗 依 然 舒 適 ， 是 機 場 接 造 最 受 歡 迎 的 服 務 車 著 。\n\n﹛_′﹛'籌7 1 穴 4 ˍ工盄 1 穴 4", {'source': 'https://i.imgur.com/QU8C5va.jpg'})
-------------------------------


光是使用內建Tesseract中文套件就可以有不錯的辨識效果。但這裡多國語文的辨識資料有大概五年沒更新了。或許接上PaddleOCR仍然可以有更好的效果。

## PaddleOCR

In [8]:
# paddlepaddle套件必須在Python 3.10才找得到
!pip install paddleocr paddlepaddle


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


嘗試把檔案下載下來之後用paddleocr辨識

In [9]:
!wget https://i.imgur.com/QU8C5va.jpg

--2023-05-31 19:19:19--  https://i.imgur.com/QU8C5va.jpg
Resolving i.imgur.com (i.imgur.com)... 151.101.40.193
Connecting to i.imgur.com (i.imgur.com)|151.101.40.193|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 182714 (178K) [image/jpeg]
Saving to: ‘QU8C5va.jpg.1’

QU8C5va.jpg.1       100%[===================>] 178.43K   543KB/s    in 0.3s    

2023-05-31 19:19:20 (543 KB/s) - ‘QU8C5va.jpg.1’ saved [182714/182714]



In [10]:
!paddleocr --image_dir QU8C5va.jpg

[2023/05/31 19:19:21] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, image_dir='QU8C5va.jpg', page_num=0, det_algorithm='DB', det_model_dir='/Users/nliang/.paddleocr/whl/det/ch/ch_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/Users/nliang/.paddleocr/whl/rec/ch/ch_PP-OCRv3_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_c

## 結論
1. 內建`Tesseract`設定使用繁體中文之後OCR結果不錯
2. `PaddleOCR`辨識結果偶爾會自己繁轉簡跑出簡體中文。例如：服務車款 -> 服务车款
3. `PaddleOCR`環境設定上比起內建的Tesseract麻煩很多。langchain內建也沒有UnstructuredPaddleImageLoader。如果要使用得要自己包裝套件使用。
4. 包裝之後的套件也無法簡單的套入`UnstructuredUrlLoader`使用。得要分別將檔案下載到本機暫存區之後傳檔案路徑使用`UnstructuredPaddleImageLoader`

綜合以上。有OCR需求的話仍然建議使用內建的`UnstructuredUrlLoader`設定`ocr_language`參數之後使用`Tesseract`做OCR

* `UnstructuredPaddleImageLoader`之實現可以參考 https://github.com/imClumsyPanda/langchain-ChatGLM/blob/master/loader/image_loader.py#L12 或本專案之`unstructed_paddle_image_loader.py`